# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

# 1º Passo: 
    Coletar as cotações do Dólar, Euro e Ouro

In [21]:
# Importando Bibliotecas que serão utilizadas #
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Abre um navegador nomeado de "robo" para ser orientado pelos comandos do Selenium #
robo = webdriver.Chrome()

# Solicito ao robo que ele faça acesso ao Google #
robo.get("https://www.google.com.br/")

# Comando que digita a pesquisa pela cotação do Dólar no Google #
robo.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Dólar")

# Solicito que seja feita a pesquisa pelo robo, digitando Enter #
robo.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Comando faz a retirada do valor correspondente à cotação do Dólar e armazena em uma variável #
cotacao_dolar = robo.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Solicito ao robo que ele faça acesso ao Google #
robo.get("https://www.google.com.br/")

# Comando que digita a pesquisa pela cotação do Euro no Google #
robo.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Euro")

# Solicito que seja feita a pesquisa pelo robo, digitando Enter #
robo.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Comando faz a retirada do valor correspondente à cotação do Euro e armazena em uma variável #
cotacao_euro = robo.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Solicito ao robo que ele faça acesso ao site de câmbio para extrair a cotação do Ouro #
robo.get("https://www.melhorcambio.com/ouro-hoje")

# Comando faz a retirada do valor correspondente à cotação do Ouro e armazena em uma variável #
cotacao_ouro = robo.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')

# Comando para formatar o valor coletado da cotação do ouro para padronizar #
cotacao_ouro = cotacao_ouro.replace(',', '.')

print(cotacao_ouro)
print(cotacao_dolar)
print(cotacao_euro)

robo.quit()



312.14
5.1093
5.544791185


### Agora vamos atualiza a nossa base de preços com as novas cotações

# 2º Passo:

- Importando a base de dados

In [23]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


# 3º Passo
    - Atualizar as cotações na base de dados

In [29]:
# Comando para  encontrar as celulas da coluna de "Moeda" que são Dolár, Euro e Ouro e atualiza o valor da cotação #
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# 4º Passo

- Atualizar os preços de compra dos produtos na base de dados

In [32]:
# Esse comando faz o recalculo do preço de compra, multiplicando preço original pelo preço das novas cotações #

tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

# 5º Passo
    - Atualizar o Preço de Venda de acordo com o novo Preço de conta e margem de venda do produto

In [34]:
# Esse comando faz o recalculo do preço de venda, multiplicando o preço da compra atualizado pelo margem #
tabela["Preço de Venda"] =  tabela["Preço de Compra"] * tabela["Margem"]

# 6º Passo
    - Exportar base atualizada

In [36]:
tabela.to_excel('Produtos Novo.xlsx', index=False)